In [1]:
%pip install semantic-link-labs

StatementMeta(, 2df180bc-3689-42ca-a6da-761d71e71d16, 7, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.7/764.7 kB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 49.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 77.8 MB/s eta 0:00:00
  Attempting uninstall: azure-core
    Found existing installation: azure-core 1.30.2
    Not uninstalling azure-core at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-781ba855-ea04-4735-bbc6-28b16b1c68db
    Can't uninstall 'azure-core'. No files were found to uninstall.
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.11.0
    Not uninstalling semantic-link-sempy at /home/trusted-service-u

In [3]:
import sempy.fabric as fabric
import sempy_labs as sempy_labs
from sempy_labs import migration, report, directlake
from sempy_labs import lakehouse as lake
from sempy_labs.tom import connect_semantic_model
import pandas as pd
import pyspark.sql.functions
from delta.tables import DeltaTable
from pyspark.sql.functions import lit, current_timestamp
from datetime import datetime
from sempy.fabric.exceptions import FabricHTTPException
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, TimestampType

StatementMeta(, 2df180bc-3689-42ca-a6da-761d71e71d16, 10, Finished, Available, Finished)

In [33]:
def fnc_LakehouseSetup(_WorkspaceID: StringType, _LH_Name: StringType):
    """
    This function will check if the specified lakehouse exists and if create it if not
    """
    Items = fabric.list_items(workspace=_WorkspaceID)
    FileInWS = Items[(Items['Type'] == 'Lakehouse') & (Items['Display Name'] == _LH_Name)]
    if(FileInWS.empty):
        try:
            lh = fabric.create_lakehouse(display_name=_LH_Name, workspace=_WorkspaceID)
            print(f"Lakehouse {_LH_Name} with ID {lh} successfully created")
        except FabricHTTPException as exc:
            print(f"An error occurred: {exc}")
    # Mount the Lakehouse for direct file system access
    lakehouse = mssparkutils.lakehouse.get(_LH_Name)
    mssparkutils.fs.mount(lakehouse.get("properties").get("abfsPath"), f"/{_LH_Name}")

StatementMeta(, 2df180bc-3689-42ca-a6da-761d71e71d16, 40, Finished, Available, Finished)

In [ ]:

WorkspaceId = fabric.get_workspace_id()



In [ ]:
def fnc_startUp(_Tablename: StringType, _LH_Name: StringType, _WS_ID: StringType):
    """
    This function will do all the initial setups for the Semantic link flow

    - Check for lakehouse and create if necessary

    - check for table in that lakehouse and truncate or drop 
    """
    fnc_LakehouseSetup(_WS_ID, _LH_Name)

    # Setup log table voor refrhes opvolging van de notebooks
    fnc_NotebookRefreshTableCheck() 

    # Truncate or drop the table to have a clear start to insert into  
    fnc_TableCheckStartOfRun(_Tablename, 'Truncate')

    # Retrieve and store local and ABFS paths of the mounted Lakehouse
    local_path = mssparkutils.fs.getMountPath(f"/{LH_Name}")
    lh_abfs_path = lakehouse.get("properties").get("abfsPath")
    return lh_abfs_path

In [ ]:
def fnc_TableCheckStartOfRun(_Tablename, _TruncateOrDrop):
    """
    Parameters:

        _Tablename = the table we are going to use

        _TruncateOrDrop = we can drop or Truncate the table, options to provide (Truncate or Drop)
    """
    if spark.catalog.tableExists(_Tablename):
        if(_TruncateOrDrop == 'Truncate'):
            spark.sql(f'TRUNCATE table {_Tablename}')
        if(_TruncateOrDrop == 'Delete'):
            spark.sql(f'DROP table {_Tablename}')


In [ ]:
def fnc_NotebookRefreshTableCheck():
    if spark.catalog.tableExists('refreshTimes_Notebooks'):
        print('refreshTimes_Notebooks Existed')
    else:    
        schema = StructType([
            StructField("NotebookName", StringType(), False),
            StructField("StartTime", TimestampType(), True),
            StructField("EndTime", TimestampType(), True)
        ])
        df = spark.createDataFrame([], schema)
        df.write.format("delta").saveAsTable('refreshTimes_Notebooks')  

In [ ]:
def fnc_PrepareColumns(_Columns):
    _Columns.columns = _Columns.columns.str.replace('[^a-zA-Z0-9]', '', regex=True)
    _Columns.columns = _Columns.columns.str.replace('[ ]', '', regex=True)
    return _Columns

In [ ]:
def fnc_LognotebookRefresh(_NotebookName: StringType, _StartTime: datetime, _EndTime: datetime):
    spark.sql(f"INSERT INTO refreshTimes_Notebooks (NotebookName, StartTime, EndTime) VALUES ('{_NotebookName}', '{_StartTime}', '{_EndTime}')")

In [ ]:
def fnc_WriteToTable(_TableDataFrame):
    """
    This function accepts a dataframe, changes the column names to something a lakehouse will accept and then save it
    """
    try:
        PreparedDataframe = fnc_PrepareColumns(_TableDataFrame)
        sparkdf = spark.createDataFrame(PreparedDataframe)
        sparkdf = sparkdf.withColumn('WSID', lit(Id))
        sparkdf.write.format("delta").option("mergeSchema", "true").mode("append").save(f"{lh_abfs_path}/Tables/{Table_Name}")
    except Exception as e:
        print(f"Error fetching Workspace objects for {Id}: {e}")
        